Merge ProintVar files with AF seq+pred and JP seq+pred:

Take merged ProIntVar files and choose columns -- save in prointvar_less \
Take those files and choose chain (found in SCOPe ID in Jpred data)

Merge ProIntVar with Alphafold -- save in AF_Pro

Add Jpred -- save in Pro_final

In [2]:
# Want to add AF sequence:

# Open prointvar files - done
# Assign the row to the sequence - done
# Alignment is correct

# Want to add JPred:

# Open Jpred final file
# Split each residue into its own column


import os
import pandas as pd
import numpy as np
import glob
from os import path
import re

os.chdir('/Users/Sophia/Dundee/dundee/data')

In [4]:
# Function to save different versions of the same pdb's:

def check_file(filePath):
    if path.exists(filePath):
        numb = 1
        while True:
            newPath = "{0}_{2}{1}".format(*path.splitext(filePath) + (numb,))
            if path.exists(newPath):
                numb += 1
            else:
                return newPath
    return filePath


In [5]:
os.chdir('/Users/Sophia/Dundee/dundee/data/ProIntVar/prointvar_merged')

x = glob.glob('*.csv')


for file in x:
    df = pd.read_csv(file)
    y = df[[
            'RES', 'CHAIN', 'AA', 
    # # #             'SS', 
            'SS_CLASS', 
                'PDB_dbResNum', 'PDB_dbResName', 'PDB_dbChainId', 'UniProt_dbAccessionId', 'UniProt_dbResNum', 'UniProt_dbResName',
                # 'PDB_codeSecondaryStructure', 'PDB_nameSecondaryStructure']]
    ]]
    y.to_csv(f'/Users/Sophia/Dundee/dundee/data/ProIntVar/prointvar_less/{file}')

/Users/Sophia/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,26,28,60,74) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:

os.chdir('/Users/Sophia/Dundee/dundee/data')
dir = os.listdir('ProIntVar/prointvar_less')

for file in dir:
    pdb = file[:4]
    # print(pdb)
    try:
        df = pd.read_csv(f'ProIntVar/prointvar_less/{file}')
        df = df.dropna()

        df_scop = pd.read_csv('Jpred/jpred_final.csv')
        df_scop = df_scop.dropna()

        df_scop['PDB_ID'] = df_scop['PDB_ID'].astype("string")

        df_trial = df_scop[df_scop['PDB_ID'].str.contains(f'{pdb}')]
        chain = df_trial.iloc[:, 2].str[5]
        chain_cap = chain.str.upper()


        df_chain = df.loc[df['PDB_dbChainId'].str.startswith(chain_cap.iloc[0])]

        filename = check_file('ProIntVar/prointvar_col_chain/%s' % file)
        df_chain.to_csv('' + filename)


    # print(df_chain)

    except AttributeError:
        print(f'Not a string value apparently: {pdb}')
        continue
    except IsADirectoryError:
        print(f'Something wrong here: {pdb}')
        continue
    except IndexError:
        print(f'Something\'s off: {pdb}')
    # except ParserError:
        # print(f'Something wrong here: {pdb}')

Something's off: 3wu2
Something's off: 4v4e
Not a string value apparently: 2cho
Not a string value apparently: 1gci
Not a string value apparently: 2erl
Not a string value apparently: 2r75


In [5]:
os.chdir('/Users/Sophia/Dundee/dundee/data')
    
    
    
data = pd.read_csv('AF/AF_m.csv')

for i,row in data.iterrows():  

    data_uni = []
    pred = []
    
    pdb = row.iloc[1] 
    pdb_id = pdb[:4]
    uniprot_seq = row.iloc[2]
    AF_pred = row.iloc[3]
    # print(pdb, pdb_id, uniprot_seq, AF_pred)
    
#     # Open Merged file - some pdb id's missing
 
    try:
        df3 = pd.read_csv('ProIntVar/prointvar_col_chain/%s.csv' % pdb_id)
    except FileNotFoundError:
        print(f'{pdb_id} is missing')
        continue
        
    # Drop NaN values
    df3_val = df3.dropna()
#     print(df3_val, pdb_id)

#   Find Uniprot offset   
        
    uniprot_offset = df3_val['UniProt_dbResNum']
    df_u = pd.DataFrame(uniprot_offset)
    df_drop = df_u.dropna()
#     print(uniprot_offset, df_u, df_drop)
    
    pos_trial = []

    for row in df_drop.iterrows():
        position_pdb = row[1][0]
#         print(position_pdb)
        try:
            pos = position_pdb.astype(int)
        except ValueError:
            print(f'Skipping {pdb_id}: PDB - Invalid data...')
            continue        

        if pos < len(uniprot_seq) and pos < len(AF_pred):
            shifted_seq = uniprot_seq[pos-1]
            shifted_pred = AF_pred[pos-1]
            data_uni.append(shifted_seq)
            pred.append(shifted_pred)
            pos_trial.append(pos)


    df_d = pd.DataFrame(data_uni)
    df_pr = pd.DataFrame(pred) 
    

    val_row = df3_val['UniProt_dbResNum'].first_valid_index()

    df_uni = df_d.shift(periods=val_row, fill_value=0)
    df_pr_2 = df_pr.shift(periods=val_row, fill_value=0)

    try:
        df3_val['shifted Uni'] = df_d
        df3_val['AF_predictions'] = df_pr

# #     print(df3_val, pdb_id)
        file = check_file(f'AF_Pro/{pdb_id}.csv')
    # Uncomment to save -- if run multiple times, saves same files with different names

        df3_val.to_csv('' + file)
    except KeyError:
        print(f'Something wrong with: {pdb}')
    except ValueError:
        print(f'Something wrong with: {pdb}')

Something wrong with: 1syx_1
Something wrong with: 1vq8_26
3wu2 is missing
Something wrong with: 1vq8_27
Something wrong with: 1vq8_25
Something wrong with: 1vq8_19
Something wrong with: 1vq8_18
Something wrong with: 1vq8
Something wrong with: 1vq8_24
Something wrong with: 1gk9
Something wrong with: 1z3e
Something wrong with: 1vq8_20
Something wrong with: 1vq8_21
Something wrong with: 1nh2_2
Something wrong with: 2jdq_1
Something wrong with: 1vq8_23
Something wrong with: 1z0j
3wu2 is missing
Something wrong with: 1vq8_22
4v4e is missing
Something wrong with: 2b59
Something wrong with: 1tx4
2cho is missing
3wu2 is missing
3wu2 is missing
Something wrong with: 1zoq
Something wrong with: 1n13
Something wrong with: 1vq8_9
Something wrong with: 1vq8_8
Something wrong with: 1vq8_6
3wu2 is missing
3wu2 is missing
Something wrong with: 1qd6
Something wrong with: 1vq8_7
1gci is missing
Something wrong with: 1upt
Something wrong with: 1vq8_5
3wu2 is missing
Something wrong with: 2grr_1
2erl is m

In [6]:
os.chdir('/Users/Sophia/Dundee/dundee/data/AF_Pro')


x = glob.glob('*.csv')

pdb_id = []
for file in x:
    pdb = file[0:4]
    pdb_id.append(pdb)
  


lol = pd.DataFrame(pdb_id)
tsp = lol.drop_duplicates()
print(len(x))
print(len(tsp))

1078
881


In [12]:
# Do the same to JPred
os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred')

# os.chdir('/cluster/gjb_lab/2524591/data')


data = pd.read_csv('jpred_final.csv')


pdb_id = []
for i,row in data.iterrows():
    pdb = row.iloc[1]
    pdb_id.append(pdb)
    scop = row.iloc[2]
        
    df_p = pd.DataFrame(pdb_id)

df_new = data['J_AA Sequence'].astype(str).apply(lambda x: pd.Series(list(x)))
df_new.dropna()

df_new['pdb_id'] = df_p
df_new


for i,row in df_new.iterrows():

    pdb_ID = row.iloc[-1]
    sh = row[:-1]
    sh = sh.dropna()

    file = check_file(f'J_seq/{pdb_ID}.csv')
    try:
        sh.to_csv('' + file)
    except ValueError:
        print('something wrong with: ' + pdb_id)

In [7]:
#  Do the same with predictions


# os.chdir('/cluster/gjb_lab/2524591/data')


data = pd.read_csv('jpred_final.csv')


pdb = []

for i,row in data.iterrows():
    pdb_id = row.iloc[1]
    
    pdb.append(pdb_id)
    
df_p = pd.DataFrame(pdb)

df_new = data['J_SS'].astype(str).apply(lambda x: pd.Series(list(x)))
df_new.dropna()

df_new['pdb_id'] = df_p
df_new


for i,row in df_new.iterrows():

    pdb_ID = row.iloc[-1]
    sh = row[:-1]

    file = check_file(f'J_ss/{pdb_ID}.csv')
    try:
        sh.to_csv('' + file)
    except ValueError:
        print('something wrong with: ' + pdb_id)


In [9]:
# Merge Jpred prediction and ss

os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred')

dir = os.listdir('J_seq')

# data = pd.read_csv('merged_w_AF/1rg8.csv')
# print(data)

for file in dir:
    try:
        df = pd.read_csv('J_seq/%s' % file)
        df_1 = pd.read_csv('J_ss/%s' % file)

        df_2 = pd.concat([df, df_1], axis=1)
        df_2 = df_2.iloc[:, [1, 3]]

        df_2.to_csv('JP/%s' % file)
    except IsADirectoryError:
        print(f'Invalid data: {file}')

In [27]:
# Check how many files and how many pdb's for JPred

os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred/J_seq')

x = glob.glob('*.csv')


os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred/J_ss')
y = glob.glob('*.csv')

os.chdir('/Users/Sophia/Dundee/dundee/data/Jpred/JP')
z = glob.glob('*.csv')

pdb_id_x = []
pdb_y = []
for file in x:
    pdb = file[0:4]
    pdb_id_x.append(pdb)
  

for file in y:
    pdb = file[0:4]
    pdb_y.append(pdb)



lol = pd.DataFrame(pdb_id_x)
rip = pd.DataFrame(pdb_y)

tsp = lol.drop_duplicates()
hm = rip.drop_duplicates()

print(len(x))
print(len(tsp))

print(len(y))
print(len(hm))

print(len(z))

1348
1235
1348
1235
1348


In [7]:
# JPred files merged with AF_Pro


os.chdir('/Users/Sophia/Dundee/dundee/data')

dir = os.listdir('AF_Pro')


for file in dir:
    try:
        df = pd.read_csv('AF_Pro/' + file)
        pdb = file[:4]
        
        file_af = check_file(f'Pro_final/{pdb}.csv')
#         if file_af == f'/homes/2524591/data/JP_shift/Pro_final/{pdb}.csv':
#         if not os.path.exists(f'/homes/2524591/data/JP_shift/Pro_final/{pdb}.csv'):
        df_j = pd.read_csv('Jpred/JP/%s.csv' % pdb)
        df_j = df_j.dropna()
        j_sel= df_j.iloc[:, 1]
        tsp = j_sel.iloc[0]
        df['Jpred_seq'] = j_sel  

        j_ss = df_j.iloc[:, 2]
        hm = j_ss.iloc[0]
        df['Jpred_SS_pred'] = j_ss
        df = df.replace('-', 'C', regex=True)
#         Put code to replace lowercase letters with C
#         df = re.sub('[a-z]', 'C', df)
        # if pdb == '1dce':
            # print(df, file_af) 
#     Uncomment to save
        df.to_csv(file_af) 

        # If there are multiple copies:
        if os.path.exists(f'Jpred/JP/{pdb}_1.csv'):
            file = check_file(f'Jpred/JP/{pdb}.csv')
            x = file[-5]
            z = int(x)
            for i in range(1, z): #len(x)??
                try:                   
                    df_j = pd.read_csv(f'Jpred/JP/{pdb}_{i}.csv')
                    df_j.dropna()
                    j_sel= df_j.iloc[:, 1]
                    tsp = j_sel.iloc[0]
                    df['Jpred_seq'] = j_sel
                                        
                    j_ss = df_j.iloc[:, 2]
                    hm = j_ss.iloc[0]
                    df['Jpred_SS_pred'] = j_ss
                    df = df.replace('-', 'C', regex=True) 
                    
                    filee = check_file(f'Pro_final/{pdb}.csv')
#                     Uncomment to save -- but if run multiple times, will save the same file with diff. name
                    df.to_csv(filee)
                except FileNotFoundError:
                    print(f'Something wrong with: {pdb}')
                    continue
                    
     
# #     except FileNotFoundError:
# #         print(f'{pdb} not found')
# #         continue
    except IsADirectoryError:
        print(f'Invalid data: {pdb}')
        continue

In [8]:
# Look at number of files
import glob
os.chdir('/Users/Sophia/Dundee/dundee/data')

# JPred
x = glob.glob('Jpred/J_ss/*.csv')
x_1 = glob.glob('Jpred/J_seq/*.csv')
y = glob.glob('Jpred/JP/*.csv')

# AF with ProIntVar
y_1 = glob.glob('AF_Pro/*.csv')

z_2 = glob.glob('Pro_final/*.csv')


print(len(x),len(x_1))
print(len(y))
print(len(y_1))
print(len(z_2))

1348 1348
1348
1078
1314
